### Przetestować uczenie się DFA ! 

In [7]:
from importlib import reload
import sys 
# sys.path.append("../utils")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../utils")
import utils.DFA, InferringDFA, utils.convDFA, InferringDFA, Inferring
reload(Inferring)
reload(InferringDFA)
reload(utils.DFA)
reload(utils.convDFA)
reload(InferringDFA)

from InferringDFA import InferringDFA
from utils.DFA import DFA
from utils.convDFA import convDFA

In [8]:
import copy
def run_learning_process(_dfa, oracle=None, debug=False):
    learn_dfa = InferringDFA(_dfa, oracle, debug=debug)
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    print(f"kontrprzykłady: {cnt_ex}, cnt = {cnt}")
    return copy.deepcopy(dfa) 

In [ ]:
_input = ['a', 'b']
dfa = DFA(Q = 2, input_signs=_input, δ= {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F = set([1]))
d = run_learning_process(copy.deepcopy(dfa), debug=False)

In [41]:
dfa = DFA(Q=3, input_signs=_input, δ= {(0, 'a'):1, (0, 'b'):0, (1, 'a'):2, (1, 'b'):1, (2, 'a'):2, (2, 'b'):2}, F= set([2]))
d = run_learning_process(dfa, debug=False)

kontrprzykłady: [], cnt = [15, 1]


In [42]:
dfa = DFA(Q=4, input_signs=_input, δ= {(0, 'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):2, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):4, (4, 'a'):4, (4, 'b'):4}, F=set([3, 4]))
d = run_learning_process(copy.deepcopy(dfa))

kontrprzykłady: [3], cnt = [24, 2]


In [43]:
dfa = DFA(Q=5, input_signs=_input, δ= {(0, 'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):2, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):4, (4, 'a'):4, (4, 'b'):4}, F=set([3]))
d = run_learning_process(copy.deepcopy(dfa), debug=False)

kontrprzykłady: [3, 7], cnt = [35, 3]


In [44]:
dfa = DFA(Q=4, input_signs=_input, δ= {(0, 'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):2, (2, 'a'):3, (2, 'b'):2, (3, 'a'):3, (3, 'b'):3}, F=set([3]))
d = run_learning_process(copy.deepcopy(dfa), debug=False)

kontrprzykłady: [3], cnt = [24, 2]


### Uczenie sie z wyrocznią **Oracle1** 
(system przepisywania słów - zamiasna sąsiednich literek)

In [4]:
_input = ['a', 'b']
from oracle1 import Oracle1

In [ ]:
"""dfa akcpetujący słowa dlugości dokłądnie 4"""
dfa_ = DFA(Q=6, input_signs=['a'], δ = {(0,'a'):1, (1,'a'):2, (2,'a'):3, (3,'a'):4, (4,'a'):5, (5,'a'):5}, 
           F = [4])
d = run_learning_process(copy.deepcopy(dfa_), oracle=None, debug=False)
# d = run_learning_process(copy.deepcopy(dfa_), oracle=Oracle1())

In [ ]:
"""dfa akcpetujący słowa dlugości dokłądnie 4"""
dfa_ = DFA(Q=6, input_signs=['a', 'b'], δ = {(0,'a'):1, (0,'b'):1, (1,'a'):2, (1,'b'):2, (2,'a'):3, (2,'b'):3, (3,'a'):4, (3,'b'):4, (4,'a'):5, (4,'b'):5, (5,'a'):5, (5,'b'):5}, 
           F = [4])
# d = run_learning_process(copy.deepcopy(dfa_), oracle=None)
d = run_learning_process(copy.deepcopy(dfa_), oracle=Oracle1())

In [ ]:
"""dfa liczacy reszte modulo 4"""
# dfa_ = DFA(Q=4, input_signs=['a'], δ = {(0,'a'):1, (1,'a'):2, (2,'a'):3, (3,'a'):0}, 
#            F = [3])
# # d = run_learning_process(copy.deepcopy(dfa_), oracle=None) # 7, 2 
# d = run_learning_process(copy.deepcopy(dfa_), oracle=Oracle1()) # 7, 2 

# dfa_ = DFA(Q=4, input_signs=['a', 'b'], δ = {(0,'a'):1, (1,'a'):2, (2,'a'):3, (3,'a'):0, (0,'b'):1, (1,'b'):2, (2,'b'):3, (3,'b'):0}, 
#            F = [3])
# # d = run_learning_process(copy.deepcopy(dfa_), oracle=None) # 24, 2 
# d = run_learning_process(copy.deepcopy(dfa_), oracle=Oracle1()) # 17, 2 

dfa_ = DFA(Q=4, input_signs=['a', 'b', 'c'], δ = {(0,'a'):1, (1,'a'):2, (2,'a'):3, (3,'a'):0, (0,'b'):1, (1,'b'):2, (2,'b'):3, (3,'b'):0, (0,'c'):1, (1,'c'):2, (2,'c'):3, (3,'c'):0}, 
           F = [3])
# d = run_learning_process(copy.deepcopy(dfa_), oracle=None) # 49, 2 
d = run_learning_process(copy.deepcopy(dfa_), oracle=Oracle1()) # 31, 2 

In [ ]:
"""dfa akceptujący słowa z conajmniej dwoma literkami 'a'"""
dfa_ = DFA(Q=3, input_signs=_input, δ = {(0, 'a'):1, (0, 'b'):0, (1, 'a'):2, (1, 'b'):1, (2, 'a'):2, (2, 'b'):2}, F=[2])
d = run_learning_process(copy.deepcopy(dfa_), oracle=None)        # cnt = [15, 1]
# d = run_learning_process(copy.deepcopy(dfa_), oracle=Oracle1()) # cnt = [12, 1]

### Uczenie się **splotu** języków 

In [4]:
import copy, sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../utils")
from importlib import reload

import Inferring, InferringDFA,  InferringconvDFA, utils.convDFA, utils.DFA
import oracle1, oracle1Conv
reload(oracle1)
reload(oracle1Conv)
reload(Inferring)
reload(InferringDFA)
reload(InferringconvDFA)
reload(utils.convDFA)
reload(utils.DFA)

from Inferring import Inferring
from InferringconvDFA import InferringconvDFA
from utils.convDFA import convDFA
from utils.DFA import DFA
from oracle1Conv import Oracle1Conv

In [5]:
def run_learning_process(_dfa, oracle=None, debug=False):
    learn_dfa = InferringconvDFA(_dfa, oracle, debug=debug)
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    print(f"kontrprzykłady: {cnt_ex}, cnt = {cnt}")
    return copy.deepcopy(dfa) 

In [13]:
"""wzorzec - 'aba'"""
dfa1 = DFA(Q = 4, input_signs=['a', 'b'], 
           δ = {(0, 'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):2, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F=set([3]))
"""wzorzec 'bbb'"""
dfa2 = DFA(Q=4, input_signs=['a', 'b'], 
           δ = {(0, 'a'):0, (0, 'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3]))

convdfa = convDFA(type="conv", dfa1=dfa1, dfa2=dfa2)
d = run_learning_process(copy.deepcopy(convdfa), oracle=None, debug=False)
print(d.route("abBBa"))

# kontrprzyklad = abBaBB
# kontrprzyklad = abBBaB
# kontrprzyklad = aBbaBB
# kontrprzyklad = aBBbaB
# kontrprzyklad = aBBBba
# kontrprzyklad = BabaBB
# kontrprzyklad = BBabaB
# kontrprzyklad = BBabaB
# kontrprzyklad = BBBaba

kontrprzyklad = abaBBB
kontrprzyklad = abBaBB
kontrprzyklad = abBBaB
kontrprzyklad = aBbaBB
kontrprzyklad = aBBbaB
kontrprzyklad = aBBBba
kontrprzyklad = BabaBB
kontrprzyklad = BBabaB
kontrprzyklad = BBBaba
kontrprzykłady: [6, 6, 6, 6, 6, 6, 6, 6, 6], cnt = [849, 10]
('abBBa', 0)


In [6]:
"""wzorzec - bb"""
dfa1 = DFA(Q=3, input_signs=['a', 'b'], 
           δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):2, (2, 'b'):2}, F= set([2]))
"""wzorzec - aa"""
dfa2 = DFA(Q=3, input_signs=['a', 'b'], 
           δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):2, (2, 'b'):2}, F= set([2]))

convdfa = convDFA(type='conv', dfa1=dfa1, dfa2=dfa2)
d = run_learning_process(copy.deepcopy(convdfa), oracle=None, debug=False)

kontrprzyklad = bbAA
kontrprzyklad = bAbA
kontrprzyklad = AbbA
kontrprzyklad = AAbb
kontrprzykłady: [4, 4, 4, 4], cnt = [289, 5]


In [10]:
"""wzorzec - bbb"""
dfa1 = DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3 }, F= set([3]))
"""wzorzec - aaa"""
dfa2 = DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F= set([3]))

print(dfa1.route("babbb"))
print(dfa2.route("aabaaa"))

convdfa = convDFA(type='conv', dfa1=dfa1, dfa2=dfa2)
d = run_learning_process(copy.deepcopy(convdfa), oracle=Oracle1Conv(), debug=False)
print(d.Q)

('babbb', 1)
('aabaaa', 1)
kontrprzyklad = bbbAAA
kontrprzyklad = bbAbAA
kontrprzyklad = bbAAbA
kontrprzyklad = bAbbAA
kontrprzyklad = bAAbbA
kontrprzyklad = bAAAbb
kontrprzyklad = AbbbAA
kontrprzyklad = AAbbbA
kontrprzyklad = AAAbbb
kontrprzykłady: [6, 6, 6, 6, 6, 6, 6, 6, 6], cnt = [333, 10]
16


In [25]:
"""słowa zawierające literke 'a'"""
dfa1 = DFA(Q=2, input_signs=['a', 'b'], δ = {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F=set([1]))
# dfa1.route("bbbb")

convdfa = convDFA(type='conv', dfa1=copy.deepcopy(dfa1), dfa2=copy.deepcopy(dfa1))
# convdfa.route("bBBAA")
d = run_learning_process(copy.deepcopy(convdfa), oracle=None, debug=False)
print(d.Q)

kontrprzykłady: [], cnt = [69, 1]
4


In [39]:
"""słowa zawierająće i literę 'a' i literę 'b'"""
dfa1 = DFA(Q=4, input_signs=['a', 'b'], δ={(0, 'a'):2, (0,'b'):1, (1, 'b'):1, (1, 'a'):3, (2, 'a'):2, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3]))

convdfa = convDFA(type='conv', dfa1 = copy.deepcopy(dfa1), dfa2 = copy.deepcopy(dfa1))
# convdfa.route("AAAbbbbaaaaaBAaa")
d = run_learning_process(copy.deepcopy(convdfa), oracle=Oracle1Conv(), debug=False)
print(d.Q)

kontrprzyklad = abAB
kontrprzyklad = aAbB
kontrprzyklad = aBbA
kontrprzyklad = baAB
kontrprzyklad = bAaB
kontrprzyklad = bBaA
kontrprzyklad = AabB
kontrprzyklad = ABab
kontrprzyklad = BabA
kontrprzykłady: [4, 4, 4, 4, 4, 4, 4, 4, 4], cnt = [402, 10]
16
